# Create Process List

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Generate Process Lists

In [ ]:
def generate_processes():
  process_num = 76 # Define the number of processes
  priority_limit = 10 # Define the max priority
  execution_limit = 30 # Define the max duration
  arrival_limit = 2000 # Define the logical arrival time limit
  deadline_min = 10 # Define the minimum deadline offset
  deadline_max = 40 # Define the maximum deadline offset
  process_id = np.arange(1,(process_num+1),1) # Define an array of process IDs, from 1 to process_num
  priority = np.random.randint(priority_limit, size=process_num) # Define an array of random priorities, from 0 (highest) to priority_limit (lowest)
  duration = np.random.randint(low=1, high=execution_limit, size=process_num) # Define an array of random durations, from 1 (lowest) to execution_limit (highest)
  arrival_time = np.random.randint(arrival_limit, size=process_num) # Define an array of random arrival times, from 0 (lowest) to arrival_limit (highest)
  start_time = np.full((process_num),fill_value=(-1),dtype=int) # Define an array of start times, for tracking purposes, all set to -1
  finish_time = np.full((process_num),fill_value=(-1),dtype=int) # Define an array of finish times for tracking purposes, all set to -1
  deadline_offset = np.random.randint(low=deadline_min,high=deadline_max, size=process_num) # Define an array of random deadline offsets, from deadline_min (lowest) to deadline_max (highest)
  deadline = np.add(duration,deadline_offset) # Define an array of deadlines, by adding the duration and deadline_offset arrays
  deadline = np.add(deadline,arrival_time) # Define an array of logical time deadlines, by adding the deadline and arrival_time arrays
  wait_time = np.full((process_num),fill_value=(-1),dtype=int) # Define an array of wait times for tracking purposes, all set to -1
  turnaround_time = np.full((process_num),fill_value=(-1),dtype=int) # Define an array of turnaround times for tracking purposes, all set to -1
  success = np.zeros((process_num),dtype=int) # Define an array of boolean successes for tracking purposes, all set to 0
  process_dict = {'Process_ID':process_id,'Priority':priority,"Duration":duration,
                  "Execution_Time_Remaining":duration,"Arrival":arrival_time,
                  "Start":start_time,"Finish":finish_time,"Deadline":deadline,
                  "Wait_Time":wait_time,"Turnaround_Time":turnaround_time,"Success":success} # Create a dictionary with the above arrays
  results_dict = {'Process_ID':[],'Priority':[],"Duration":[],"Execution_Time_Remaining":[],"Arrival":[],"Start":[],"Finish":[],"Deadline":[],"Wait_Time":[],"Turnaround_Time":[],"Success":[]} # Create an empty dictionary for tracking purposes, which has the same keys as the above but no values
  process_df = pd.DataFrame.from_dict(process_dict) # Convert the process dict to a Pandas dataframe
  results_df = pd.DataFrame.from_dict(results_dict) # Convert the results dict to a Pandas dataframe
  int_columns = ['Priority',"Duration","Execution_Time_Remaining","Arrival","Finish","Deadline","Start","Wait_Time","Turnaround_Time","Success"] # Define the set of columns that should all be integers
  process_df[int_columns] = process_df[int_columns].astype(int) # Apply the integer dtype to the above columns in the process dictionary
  process_df['Process_ID'] = process_df['Process_ID'].astype(object)  # Set the Process_ID column in the process dictionary to be an object, as it should be a string and not a number
  results_df[int_columns] = results_df[int_columns].astype(int) # Apply the integer dtype to the above columns in the results dictionary
  results_df['Process_ID'] = results_df['Process_ID'].astype(object) # Set the Process_ID column in the results dictionary to be an object, as it should be a string and not a number
  return process_df, results_df # Return the process and results dataframes

## Process Generation Example

In [ ]:
process_df, results_df = generate_processes()
process_df

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
0,1,9,2,2,1805,-1,-1,1835,-1,-1,0
1,2,8,2,2,1526,-1,-1,1559,-1,-1,0
2,3,1,13,13,33,-1,-1,59,-1,-1,0
3,4,2,19,19,1633,-1,-1,1662,-1,-1,0
4,5,0,7,7,464,-1,-1,483,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...
71,72,3,17,17,1950,-1,-1,1978,-1,-1,0
72,73,7,7,7,1999,-1,-1,2041,-1,-1,0
73,74,9,18,18,1296,-1,-1,1335,-1,-1,0
74,75,8,19,19,1969,-1,-1,1999,-1,-1,0


In [ ]:
results_df.dtypes

Process_ID                  object
Priority                     int64
Duration                     int64
Execution_Time_Remaining     int64
Arrival                      int64
Start                        int64
Finish                       int64
Deadline                     int64
Wait_Time                    int64
Turnaround_Time              int64
Success                      int64
dtype: object

## Create a Transient Process Dataframe

In [ ]:
transient_dict = {'Process_ID':['A','B','C','D','E','F','G','H','I','J'],
                  'Priority':[0,1,2,3,5,4,6,7,8,9],
                  "Duration":[5,10,20,15,25,30,35,40,45,50],
                  "Execution_Time_Remaining":[5,10,20,15,25,20,35,40,45,50],
                  "Arrival":[100,200,300,300,500,500,700,800,900,1000],
                  "Start":[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
                  "Finish":[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
                  "Deadline":[110,220,330,340,550,560,770,880,990,1100],
                  "Wait_Time":[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
                  "Turnaround_Time":[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
                  "Success":[0,0,0,0,0,0,0,0,0,0]} # Manually define a dictionary, like above
transient_df = pd.DataFrame.from_dict(transient_dict) # Create a new transient dataframe based on that dictionary
transient_df

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
0,A,0,5,5,100,-1,-1,110,-1,-1,0
1,B,1,10,10,200,-1,-1,220,-1,-1,0
2,C,2,20,20,300,-1,-1,330,-1,-1,0
3,D,3,15,15,300,-1,-1,340,-1,-1,0
4,E,5,25,25,500,-1,-1,550,-1,-1,0
5,F,4,30,20,500,-1,-1,560,-1,-1,0
6,G,6,35,35,700,-1,-1,770,-1,-1,0
7,H,7,40,40,800,-1,-1,880,-1,-1,0
8,I,8,45,45,900,-1,-1,990,-1,-1,0
9,J,9,50,50,1000,-1,-1,1100,-1,-1,0


# First-Come, First-Served (FCFS or FIFO)

In [ ]:
def fcfs(adhoc=None,poll=15):
  i = 0 # Initialize a process counter to track successfully executed processes
  time = 0 # Initialize the logical clock counter at 0
  process_df, results_df = generate_processes() # Generate a list of processes
  if adhoc is not None: # If a transient/adhoc/manually-defined list of processes was provided:
      process_df = pd.concat([process_df,adhoc], ignore_index=True) # Combine the randomly-generated and manually-defined process lists
  new_process_list = process_df.loc[(process_df['Arrival'] > (time - poll)) & (process_df['Arrival'] <= time)] # Define the first set of processes to schedule, which should have an arrival time less than or equal to the current time (0) and greater than (current time - polling rate)
  first_list_df = new_process_list.sort_values(by=['Arrival'], ascending=[True]) # Take the set of new processes and sort them, in this case according to arrival time (ties are won by whichever process occurs first in the dataframe)
  while i < len(process_df): # As long as not all the processes have been executed:
    if first_list_df.empty == False: # As long as the current executing process list is not empty:
      for process in first_list_df['Process_ID']: # For each process in the process list:
        execution = first_list_df.Execution_Time_Remaining[first_list_df['Process_ID'] == process].item() # Copy the execution time
        if first_list_df[first_list_df['Process_ID'] == process]['Start'].item() == -1: # If the execution time is -1 (we're using -1 here because we start at time 0, so it's possible for a process that arrives right at 0 to be set more than once):
          first_list_df.Start[first_list_df['Process_ID'] == process] = time # Set the start time
          first_list_df.Wait_Time[first_list_df['Process_ID'] == process] = (time - first_list_df.Arrival[first_list_df['Process_ID'] == process].item()) # Set the wait time, which is the difference between the start and arrival times
        while execution > 0: # As long as the process has any logical execution time remaining:
          execution -= 1 # Reduce the time remaning by 1
          time += 1 # Increase the current logical time by 1
          if (time % poll == 0): # If <poll> cycles have past (the polling cycle timer "expires"):
            first_list_df.Execution_Time_Remaining[first_list_df['Process_ID'] == process] = execution # Save the logical execution time remaining of the currently executing process
            new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)] # Retrieve the next set of processes based on their arrival times, if any
            first_list_df = pd.concat([first_list_df,new_process_list]) # Combine the current process list with the new process list
            first_list_df = first_list_df.sort_values(by=['Arrival'], ascending=[True]) # Sort the combined list, in this case by arrival time (which should result in no changes)
            break # Break out of the current inner for loop, to return to the outer while loop and pick up the next process (which for FCFS, should be the process that was previously executing)
        else: # If the process's execution time remaining is <= 0 (which should only ever be == 0), it has "successfully processed", so:
          first_list_df.Execution_Time_Remaining[first_list_df['Process_ID'] == process] = process_df.Execution_Time_Remaining[process_df['Process_ID'] == process].item() # Set the execution time remaining BACK to the original time remaining, for tracking purposes
          first_list_df.Finish[first_list_df['Process_ID'] == process] = time # Set the finish time to the current time
          first_list_df.Turnaround_Time[first_list_df['Process_ID'] == process] = (time - first_list_df.Arrival[first_list_df['Process_ID'] == process].item()) # Set the turnaround time, which is the difference between the finish time and the logical arrival time
          if time <= first_list_df.Deadline[first_list_df['Process_ID'] == process].item(): # If the process finished executing prior to its deadline:
              first_list_df.Success[first_list_df['Process_ID'] == process] = 1 # Set the Success flag to 1 (True)
          results_df = pd.concat([results_df,first_list_df.loc[first_list_df.Process_ID.eq(process)]]) # Add this process to the results dataframe
          first_list_df.drop(index=first_list_df[first_list_df['Process_ID'] == process].index, inplace = True) # Delete this process from the executing process list
          i += 1 # Increase the process counter
        break # Break out of the inner for loop back to the outer while loop, so that the while loop can pick up the next process
    else: # If the current executing process list is empty:
      time += 1 # Increae the current logical time
      if (time % poll == 0): # If <poll> cycles have past (the polling cycle timer "expires"):
        new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)] # Retrieve the next set of processes based on their arrival times, if any
        first_list_df = new_process_list.sort_values(by=['Arrival'], ascending=[True]) # Sort the combined list, in this case by arrival time (please note that no concat happens here, as there is currently nothing to add this to)
  return results_df # Return the results dataframe

In [ ]:
fcfs_results = fcfs(adhoc=transient_df)
fcfs_results

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
57,58,1,15,15,3,15,30,44,12,27,1
70,71,4,21,21,82,90,111,131,8,29,1
24,25,9,28,28,95,111,139,146,16,44,1
76,A,0,5,5,100,139,144,110,39,44,0
65,66,2,16,16,143,150,166,179,7,23,1
...,...,...,...,...,...,...,...,...,...,...,...
3,4,4,8,8,1793,1818,1826,1820,25,33,0
35,36,7,17,17,1827,1830,1847,1874,3,20,1
68,69,2,28,28,1897,1905,1933,1948,8,36,1
43,44,1,11,11,1917,1933,1944,1944,16,27,1


In [ ]:
print(list(fcfs_results['Process_ID']))
print(f'Wait times: {list(fcfs_results["Wait_Time"])}')
print(f'Average Wait times: {np.mean(list(fcfs_results["Wait_Time"]))}')
print(f'Turnaround times: {list(fcfs_results["Turnaround_Time"])}')
print(f'Average Turnaround times: {np.mean(list(fcfs_results["Turnaround_Time"]))}')
print(f'Successes: {list(fcfs_results["Success"])}')
print(f'Average Successes: {np.mean(list(fcfs_results["Success"]))}')

[58, 71, 25, 'A', 66, 60, 'B', 23, 15, 'C', 'D', 7, 20, 74, 38, 21, 2, 33, 41, 'E', 'F', 9, 43, 48, 35, 40, 49, 50, 42, 'G', 28, 63, 27, 22, 19, 51, 8, 'H', 55, 14, 30, 'I', 70, 59, 65, 52, 39, 62, 12, 'J', 1, 72, 5, 37, 16, 31, 57, 24, 29, 68, 6, 18, 73, 54, 45, 67, 47, 3, 75, 46, 61, 64, 34, 13, 26, 11, 32, 76, 56, 17, 53, 4, 36, 69, 44, 10]
Wait times: [12, 8, 16, 39, 7, 10, 21, 6, 3, 18, 38, 48, 35, 33, 16, 0, 8, 18, 13, 13, 38, 57, 27, 31, 31, 41, 11, 14, 22, 6, 39, 53, 35, 48, 47, 45, 50, 33, 36, 13, 19, 8, 46, 66, 72, 85, 72, 87, 86, 82, 62, 71, 21, 35, 51, 7, 6, 14, 7, 11, 5, 25, 28, 5, 5, 14, 22, 39, 60, 10, 33, 35, 28, 37, 61, 83, 49, 61, 65, 43, 10, 25, 3, 8, 16, 12]
Average Wait times: 31.732558139534884
Turnaround times: [27, 29, 44, 44, 23, 21, 31, 18, 21, 38, 53, 69, 39, 59, 17, 17, 28, 33, 16, 38, 58, 70, 47, 35, 45, 65, 12, 23, 29, 41, 64, 63, 49, 56, 59, 64, 54, 73, 39, 29, 35, 53, 68, 87, 88, 108, 89, 109, 94, 132, 82, 81, 43, 52, 74, 8, 19, 24, 32, 26, 32, 28, 47, 2

In [ ]:
fcfs_results.loc[(fcfs_results['Arrival'] < 150) |
                  ((fcfs_results['Arrival'] >= 200) & (fcfs_results['Arrival'] < 250)) |
                  ((fcfs_results['Arrival'] >= 300) & (fcfs_results['Arrival'] < 350)) |
                  ((fcfs_results['Arrival'] >= 500) & (fcfs_results['Arrival'] < 550)) |
                  ((fcfs_results['Arrival'] >= 700) & (fcfs_results['Arrival'] < 750)) |
                  ((fcfs_results['Arrival'] >= 800) & (fcfs_results['Arrival'] < 850)) |
                  ((fcfs_results['Arrival'] >= 900) & (fcfs_results['Arrival'] < 950)) |
                  ((fcfs_results['Arrival'] >= 1000) & (fcfs_results['Arrival'] < 1050))]

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
57,58,1,15,15,3,15,30,44,12,27,1
70,71,4,21,21,82,90,111,131,8,29,1
24,25,9,28,28,95,111,139,146,16,44,1
76,A,0,5,5,100,139,144,110,39,44,0
65,66,2,16,16,143,150,166,179,7,23,1
59,60,9,11,11,200,210,221,238,10,21,1
77,B,1,10,10,200,221,231,220,21,31,0
22,23,7,12,12,234,240,252,276,6,18,1
78,C,2,20,20,300,318,338,330,18,38,0
79,D,3,15,15,300,338,353,340,38,53,0


# Priority Scheduling

In [ ]:
def priority(adhoc=None,poll=15): # PLEASE NOTE: as these five algorithms are largely identical, going forward only the lines of code that are different will be commented.  Please refer to the FCFS method if any lines are not clear.
  i = 0
  time = 0
  process_df, results_df = generate_processes()
  if adhoc is not None:
      process_df = pd.concat([process_df,adhoc], ignore_index=True)
  new_process_list = process_df.loc[(process_df['Arrival'] > (time - poll)) & (process_df['Arrival'] <= time)]
  priority_list_df = new_process_list.sort_values(by=['Priority','Arrival'], ascending=[True,True]) # Take the set of new processes and sort them, in this case according to Priority, with 0 being the highest (ties are won by whichever process arrived first)
  while i < len(process_df):
    if priority_list_df.empty == False:
      for process in priority_list_df['Process_ID']:
        execution = priority_list_df.Execution_Time_Remaining[priority_list_df['Process_ID'] == process].item()
        if priority_list_df[priority_list_df['Process_ID'] == process]['Start'].item() == -1:
          priority_list_df.Start[priority_list_df['Process_ID'] == process] = time
          priority_list_df.Wait_Time[priority_list_df['Process_ID'] == process] = (time - priority_list_df.Arrival[priority_list_df['Process_ID'] == process].item())
        while execution > 0:
          execution -= 1
          time += 1
          if (time % poll == 0):
            priority_list_df.Execution_Time_Remaining[priority_list_df['Process_ID'] == process] = execution
            new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)]
            priority_list_df = pd.concat([priority_list_df,new_process_list])
            priority_list_df = priority_list_df.sort_values(by=['Priority','Arrival'], ascending=[True,True]) # Take the new combined list and sort it, in this case according to Priority, with 0 being the highest (ties are won by whichever process arrived first)
            break
        else:
          priority_list_df.Execution_Time_Remaining[priority_list_df['Process_ID'] == process] = process_df.Execution_Time_Remaining[process_df['Process_ID'] == process].item()
          priority_list_df.Finish[priority_list_df['Process_ID'] == process] = time
          priority_list_df.Turnaround_Time[priority_list_df['Process_ID'] == process] = (time - priority_list_df.Arrival[priority_list_df['Process_ID'] == process].item())
          if time <= priority_list_df.Deadline[priority_list_df['Process_ID'] == process].item():
              priority_list_df.Success[priority_list_df['Process_ID'] == process] = 1
          results_df = pd.concat([results_df,priority_list_df.loc[priority_list_df.Process_ID.eq(process)]])
          priority_list_df.drop(index=priority_list_df[priority_list_df['Process_ID'] == process].index, inplace = True)
          i += 1
        break
    else:
      time += 1
      if (time % poll == 0):
        new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)]
        priority_list_df = new_process_list.sort_values(by=['Priority','Arrival'], ascending=[True,True]) # Take the set of new processes and sort them, in this case according to Priority, with 0 being the highest (ties are won by whichever process arrived first)
  return results_df

In [ ]:
priority_results = priority(adhoc=transient_df)
priority_results

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
60,61,8,19,19,41,45,64,73,4,23,1
17,18,8,16,16,74,75,91,109,1,17,1
76,A,0,5,5,100,105,110,110,5,10,1
32,33,6,4,4,96,110,114,115,14,18,1
65,66,5,7,7,115,120,127,137,5,12,1
...,...,...,...,...,...,...,...,...,...,...,...
52,53,4,3,3,1962,1965,1968,1988,3,6,1
58,59,6,15,15,1962,1968,1983,1997,6,21,1
67,68,6,7,7,1977,1983,1990,2002,6,13,1
31,32,7,23,23,1975,1990,2013,2020,15,38,1


In [ ]:
print(list(priority_results['Process_ID']))
print(f'Wait times: {list(priority_results["Wait_Time"])}')
print(f'Average Wait times: {np.mean(list(priority_results["Wait_Time"]))}')
print(f'Turnaround times: {list(priority_results["Turnaround_Time"])}')
print(f'Average Turnaround times: {np.mean(list(priority_results["Turnaround_Time"]))}')
print(f'Successes: {list(priority_results["Success"])}')
print(f'Average Successes: {np.mean(list(priority_results["Success"]))}')

[61, 18, 'A', 33, 66, 13, 10, 'B', 52, 9, 36, 74, 'C', 55, 'D', 12, 58, 20, 24, 67, 26, 40, 5, 31, 46, 'F', 42, 'E', 38, 16, 4, 65, 1, 7, 43, 'G', 28, 'H', 23, 25, 'I', 57, 73, 47, 48, 54, 56, 3, 60, 71, 69, 17, 30, 'J', 62, 15, 51, 8, 70, 75, 6, 35, 14, 44, 45, 11, 63, 39, 34, 49, 27, 19, 29, 2, 22, 50, 64, 76, 41, 37, 72, 53, 59, 68, 32, 21]
Wait times: [4, 1, 5, 14, 5, 12, 20, 10, 6, 9, 8, 1, 7, 10, 27, 3, 2, 24, 18, 42, 34, 26, 4, 7, 6, 29, 14, 49, 17, 24, 5, 282, 7, 6, 11, 5, 3, 10, 11, 11, 0, 2, 7, 4, 2, 21, 21, 11, 18, 4, 12, 10, 29, 48, 10, 14, 10, 12, 23, 29, 12, 60, 1, 6, 2, 6, 2, 17, 3, 16, 8, 23, 49, 13, 9, 46, 12, 5, 9, 5, 14, 3, 6, 6, 15, 56]
Average Wait times: 17.209302325581394
Turnaround times: [23, 17, 10, 18, 12, 41, 28, 20, 12, 11, 16, 8, 27, 29, 61, 30, 17, 29, 83, 59, 40, 42, 10, 34, 55, 49, 27, 87, 148, 26, 21, 310, 27, 9, 33, 62, 25, 50, 31, 36, 70, 11, 103, 19, 19, 57, 83, 32, 43, 8, 35, 18, 31, 181, 34, 19, 39, 90, 48, 150, 17, 72, 27, 28, 8, 10, 17, 39, 20, 

In [ ]:
priority_results.loc[(priority_results['Arrival'] < 150) |
                  ((priority_results['Arrival'] >= 200) & (priority_results['Arrival'] < 250)) |
                  ((priority_results['Arrival'] >= 300) & (priority_results['Arrival'] < 350)) |
                  ((priority_results['Arrival'] >= 500) & (priority_results['Arrival'] < 550)) |
                  ((priority_results['Arrival'] >= 700) & (priority_results['Arrival'] < 750)) |
                  ((priority_results['Arrival'] >= 800) & (priority_results['Arrival'] < 850)) |
                  ((priority_results['Arrival'] >= 900) & (priority_results['Arrival'] < 950)) |
                  ((priority_results['Arrival'] >= 1000) & (priority_results['Arrival'] < 1050))]

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
60,61,8,19,19,41,45,64,73,4,23,1
17,18,8,16,16,74,75,91,109,1,17,1
76,A,0,5,5,100,105,110,110,5,10,1
32,33,6,4,4,96,110,114,115,14,18,1
65,66,5,7,7,115,120,127,137,5,12,1
12,13,6,29,29,138,150,179,203,12,41,1
77,B,1,10,10,200,210,220,220,10,20,1
51,52,3,6,6,219,225,231,258,6,12,1
8,9,5,2,2,222,231,233,241,9,11,1
78,C,2,20,20,300,307,327,330,7,27,1


# Shortest-Job First (SJF)

In [ ]:
def sjf(adhoc=None,poll=15): # PLEASE NOTE: as these five algorithms are largely identical, going forward only the lines of code that are different will be commented.  Please refer to the FCFS method if any lines are not clear.
  i = 0
  time = 0
  process_df, results_df = generate_processes()
  if adhoc is not None:
      process_df = pd.concat([process_df,adhoc], ignore_index=True)
  new_process_list = process_df.loc[(process_df['Arrival'] > (time - poll)) & (process_df['Arrival'] <= time)]
  shortest_list_df = new_process_list.sort_values(by=['Duration','Arrival'], ascending=[True,True]) # Take the set of new processes and sort them, in this case according to Duration (ties are won by whichever process arrived first)
  while i < len(process_df):
    if shortest_list_df.empty == False:
      for process in shortest_list_df['Process_ID']:
        execution = shortest_list_df.Execution_Time_Remaining[shortest_list_df['Process_ID'] == process].item()
        if shortest_list_df[shortest_list_df['Process_ID'] == process]['Start'].item() == -1:
          shortest_list_df.Start[shortest_list_df['Process_ID'] == process] = time
          shortest_list_df.Wait_Time[shortest_list_df['Process_ID'] == process] = (time - shortest_list_df.Arrival[shortest_list_df['Process_ID'] == process].item())
        while execution > 0:
          execution -= 1
          time += 1
          if (time % poll == 0):
            shortest_list_df.Execution_Time_Remaining[shortest_list_df['Process_ID'] == process] = execution
            new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)]
            shortest_list_df = pd.concat([shortest_list_df,new_process_list])
            shortest_list_df = shortest_list_df.sort_values(by=['Duration','Arrival'], ascending=[True,True]) # Take the combined list and sort it, in this case according to Duration (ties are won by whichever process arrived first)
            break
        else:
          shortest_list_df.Execution_Time_Remaining[shortest_list_df['Process_ID'] == process] = process_df.Execution_Time_Remaining[process_df['Process_ID'] == process].item()
          shortest_list_df.Finish[shortest_list_df['Process_ID'] == process] = time
          shortest_list_df.Turnaround_Time[shortest_list_df['Process_ID'] == process] = (time - shortest_list_df.Arrival[shortest_list_df['Process_ID'] == process].item())
          if time <= shortest_list_df.Deadline[shortest_list_df['Process_ID'] == process].item():
              shortest_list_df.Success[shortest_list_df['Process_ID'] == process] = 1
          results_df = pd.concat([results_df,shortest_list_df.loc[shortest_list_df.Process_ID.eq(process)]])
          shortest_list_df.drop(index=shortest_list_df[shortest_list_df['Process_ID'] == process].index, inplace = True)
          i += 1
        break
    else:
      time += 1
      if (time % poll == 0):
        new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)]
        shortest_list_df = new_process_list.sort_values(by=['Duration','Arrival'], ascending=[True,True]) # Take the set of new processes and sort them, in this case according to Duration (ties are won by whichever process arrived first)
  return results_df

In [ ]:
sjf_results = sjf(adhoc=transient_df)
sjf_results

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
1,2,4,5,5,7,15,20,26,8,13,1
48,49,9,25,25,12,20,45,52,8,33,1
76,A,0,5,5,100,105,110,110,5,10,1
56,57,4,5,5,127,135,140,156,8,13,1
22,23,6,10,10,134,140,150,166,6,16,1
...,...,...,...,...,...,...,...,...,...,...,...
60,61,7,29,29,1843,1852,1881,1904,9,38,1
14,15,4,25,25,1894,1905,1930,1930,11,36,1
57,58,5,21,21,1927,1935,1956,1964,8,29,1
47,48,0,24,24,1966,1980,2004,2003,14,38,0


In [ ]:
print(list(sjf_results['Process_ID']))
print(f'Wait times: {list(sjf_results["Wait_Time"])}')
print(f'Average Wait times: {np.mean(list(sjf_results["Wait_Time"]))}')
print(f'Turnaround times: {list(sjf_results["Turnaround_Time"])}')
print(f'Average Turnaround times: {np.mean(list(sjf_results["Turnaround_Time"]))}')
print(f'Successes: {list(sjf_results["Success"])}')
print(f'Average Successes: {np.mean(list(sjf_results["Success"]))}')

[2, 49, 'A', 57, 23, 60, 65, 'B', 25, 63, 72, 'D', 22, 'C', 44, 36, 71, 64, 69, 41, 29, 14, 5, 'E', 47, 'F', 19, 74, 18, 3, 33, 56, 20, 51, 'G', 8, 73, 16, 'H', 12, 7, 'I', 39, 24, 50, 'J', 4, 10, 6, 37, 1, 43, 42, 76, 17, 67, 54, 55, 34, 45, 11, 30, 38, 70, 46, 75, 62, 26, 68, 66, 28, 59, 13, 21, 35, 32, 53, 27, 31, 40, 52, 61, 15, 58, 48, 9]
Wait times: [8, 8, 5, 8, 6, 11, 2, 10, 22, 11, 23, 0, 14, 15, 7, 0, 9, 13, 13, 3, 16, 14, 30, 35, 5, 135, 9, 9, 14, 15, 0, 72, 41, 13, 109, 13, 4, 4, 48, 5, 11, 20, 5, 5, 1, 29, 11, 12, 16, 14, 1, 9, 8, 5, 10, 0, 14, 4, 119, 6, 4, 6, 41, 164, 8, 74, 11, 4, 291, 8, 10, 7, 234, 422, 385, 361, 11, 7, 1, 7, 7, 9, 11, 8, 14, 15]
Average Wait times: 37.19767441860465
Turnaround times: [13, 33, 10, 13, 16, 44, 27, 20, 50, 16, 95, 15, 24, 45, 30, 8, 27, 17, 38, 11, 25, 36, 76, 123, 145, 155, 15, 30, 35, 60, 22, 121, 69, 17, 148, 16, 14, 23, 120, 6, 33, 88, 13, 29, 9, 87, 21, 22, 32, 30, 64, 29, 50, 135, 21, 28, 28, 14, 196, 240, 20, 26, 64, 250, 34, 127,

In [ ]:
sjf_results.loc[(sjf_results['Arrival'] < 150) |
                  ((sjf_results['Arrival'] >= 200) & (sjf_results['Arrival'] < 250)) |
                  ((sjf_results['Arrival'] >= 300) & (sjf_results['Arrival'] < 350)) |
                  ((sjf_results['Arrival'] >= 500) & (sjf_results['Arrival'] < 550)) |
                  ((sjf_results['Arrival'] >= 700) & (sjf_results['Arrival'] < 750)) |
                  ((sjf_results['Arrival'] >= 800) & (sjf_results['Arrival'] < 850)) |
                  ((sjf_results['Arrival'] >= 900) & (sjf_results['Arrival'] < 950)) |
                  ((sjf_results['Arrival'] >= 1000) & (sjf_results['Arrival'] < 1050))]

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
1,2,4,5,5,7,15,20,26,8,13,1
48,49,9,25,25,12,20,45,52,8,33,1
76,A,0,5,5,100,105,110,110,5,10,1
56,57,4,5,5,127,135,140,156,8,13,1
22,23,6,10,10,134,140,150,166,6,16,1
59,60,2,18,18,109,120,153,145,11,44,0
77,B,1,10,10,200,210,220,220,10,20,1
79,D,3,15,15,300,300,315,340,0,15,1
21,22,2,10,10,316,330,340,342,14,24,1
78,C,2,20,20,300,315,345,330,15,45,0


# Least Execution Time Remaining

In [ ]:
def least(adhoc=None,poll=15): # PLEASE NOTE: as these five algorithms are largely identical, going forward only the lines of code that are different will be commented.  Please refer to the FCFS method if any lines are not clear.
  i = 0
  time = 0
  process_df, results_df = generate_processes()
  if adhoc is not None:
      process_df = pd.concat([process_df,adhoc], ignore_index=True)
  new_process_list = process_df.loc[(process_df['Arrival'] > (time - poll)) & (process_df['Arrival'] <= time)]
  least_list_df = new_process_list.sort_values(by=['Execution_Time_Remaining','Arrival'], ascending=[True,True]) # Take the set of new processes and sort them, in this case according to Execution Time Remaining (ties are won by whichever process arrived first)
  while i < len(process_df):
    if least_list_df.empty == False:
      for process in least_list_df['Process_ID']:
        execution = least_list_df.Execution_Time_Remaining[least_list_df['Process_ID'] == process].item()
        if least_list_df[least_list_df['Process_ID'] == process]['Start'].item() == -1:
          least_list_df.Start[least_list_df['Process_ID'] == process] = time
          least_list_df.Wait_Time[least_list_df['Process_ID'] == process] = (time - least_list_df.Arrival[least_list_df['Process_ID'] == process].item())
        while execution > 0:
          execution -= 1
          time += 1
          if (time % poll == 0):
            least_list_df.Execution_Time_Remaining[least_list_df['Process_ID'] == process] = execution
            new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)]
            least_list_df = pd.concat([least_list_df,new_process_list])
            least_list_df = least_list_df.sort_values(by=['Execution_Time_Remaining','Arrival'], ascending=[True,True]) # Take the combined process list and sort it, in this case according to Execution Time Remaining (ties are won by whichever process arrived first)
            break
        else:
          least_list_df.Execution_Time_Remaining[least_list_df['Process_ID'] == process] = process_df.Execution_Time_Remaining[process_df['Process_ID'] == process].item()
          least_list_df.Finish[least_list_df['Process_ID'] == process] = time
          least_list_df.Turnaround_Time[least_list_df['Process_ID'] == process] = (time - least_list_df.Arrival[least_list_df['Process_ID'] == process].item())
          if time <= least_list_df.Deadline[least_list_df['Process_ID'] == process].item():
              least_list_df.Success[least_list_df['Process_ID'] == process] = 1
          results_df = pd.concat([results_df,least_list_df.loc[least_list_df.Process_ID.eq(process)]])
          least_list_df.drop(index=least_list_df[least_list_df['Process_ID'] == process].index, inplace = True)
          i += 1
        break
    else:
      time += 1
      if (time % poll == 0):
        new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)]
        least_list_df = new_process_list.sort_values(by=['Execution_Time_Remaining','Arrival'], ascending=[True,True]) # Take the set of new processes and sort them, in this case according to Execution Time Remaining (ties are won by whichever process arrived first)
  return results_df

In [ ]:
least_results = least(adhoc=transient_df)
least_results

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
2,3,4,15,15,28,30,45,80,2,17,1
18,19,2,7,7,88,90,97,113,2,9,1
76,A,0,5,5,100,105,110,110,5,10,1
46,47,1,26,26,144,150,176,203,6,32,1
31,32,7,4,4,204,210,214,225,6,10,1
...,...,...,...,...,...,...,...,...,...,...,...
10,11,9,7,7,1776,1785,1792,1812,9,16,1
45,46,2,21,21,1833,1845,1866,1864,12,33,0
64,65,4,14,14,1856,1866,1880,1882,10,24,1
4,5,6,20,20,1874,1880,1900,1928,6,26,1


In [ ]:
print(list(least_results['Process_ID']))
print(f'Wait times: {list(least_results["Wait_Time"])}')
print(f'Average Wait times: {np.mean(list(least_results["Wait_Time"]))}')
print(f'Turnaround times: {list(least_results["Turnaround_Time"])}')
print(f'Average Turnaround times: {np.mean(list(least_results["Turnaround_Time"]))}')
print(f'Successes: {list(least_results["Success"])}')
print(f'Average Successes: {np.mean(list(least_results["Success"]))}')

[3, 19, 'A', 47, 32, 21, 'B', 48, 71, 37, 25, 'D', 27, 'C', 69, 43, 40, 76, 22, 23, 15, 7, 45, 31, 33, 'F', 68, 60, 'E', 38, 64, 13, 58, 67, 1, 54, 61, 9, 26, 20, 53, 59, 39, 'G', 2, 18, 16, 4, 35, 70, 73, 'H', 74, 63, 51, 57, 17, 12, 49, 'I', 29, 41, 55, 8, 34, 72, 'J', 24, 66, 36, 44, 62, 14, 42, 52, 75, 28, 30, 50, 56, 10, 11, 46, 65, 5, 6]
Wait times: [2, 2, 5, 6, 6, 17, 21, 9, 13, 15, 23, 13, 12, 28, 13, 56, 9, 21, 9, 18, 1, 17, 19, 12, 14, 21, 26, 77, 98, 48, 3, 0, 13, 21, 11, 13, 14, 8, 292, 30, 13, 14, 13, 174, 0, 7, 13, 25, 7, 7, 4, 109, 38, 12, 17, 76, 7, 5, 14, 224, 8, 11, 15, 3, 28, 2, 220, 13, 1, 38, 10, 10, 9, 17, 6, 7, 9, 8, 4, 11, 18, 9, 12, 10, 6, 9]
Average Wait times: 26.848837209302324
Turnaround times: [17, 9, 10, 32, 10, 24, 31, 17, 28, 24, 34, 28, 15, 51, 26, 97, 14, 31, 24, 42, 6, 24, 45, 38, 28, 55, 47, 99, 123, 76, 11, 5, 52, 42, 25, 30, 29, 48, 321, 59, 24, 21, 36, 209, 14, 8, 23, 64, 14, 25, 10, 233, 63, 26, 54, 105, 25, 13, 24, 305, 23, 38, 27, 7, 46, 17, 3

In [ ]:
least_results.loc[(least_results['Arrival'] < 150) |
                  ((least_results['Arrival'] >= 200) & (least_results['Arrival'] < 250)) |
                  ((least_results['Arrival'] >= 300) & (least_results['Arrival'] < 350)) |
                  ((least_results['Arrival'] >= 500) & (least_results['Arrival'] < 550)) |
                  ((least_results['Arrival'] >= 700) & (least_results['Arrival'] < 750)) |
                  ((least_results['Arrival'] >= 800) & (least_results['Arrival'] < 850)) |
                  ((least_results['Arrival'] >= 900) & (least_results['Arrival'] < 950)) |
                  ((least_results['Arrival'] >= 1000) & (least_results['Arrival'] < 1050))]

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
2,3,4,15,15,28,30,45,80,2,17,1
18,19,2,7,7,88,90,97,113,2,9,1
76,A,0,5,5,100,105,110,110,5,10,1
46,47,1,26,26,144,150,176,203,6,32,1
31,32,7,4,4,204,210,214,225,6,10,1
77,B,1,10,10,200,221,231,220,21,31,0
79,D,3,15,15,300,313,328,340,13,28,1
26,27,8,3,3,318,330,333,337,12,15,1
78,C,2,20,20,300,328,351,330,28,51,0
68,69,0,13,13,347,360,373,388,13,26,1


# Round Robin (RR)

In [ ]:
def rr(adhoc=None,poll=15): # PLEASE NOTE: as these five algorithms are largely identical, going forward only the lines of code that are different will be commented.  Please refer to the FCFS method if any lines are not clear.
  i = 0
  time = 0
  process_df, results_df = generate_processes()
  if adhoc is not None:
      process_df = pd.concat([process_df,adhoc], ignore_index=True)
  new_process_list = process_df.loc[(process_df['Arrival'] > (time - poll)) & (process_df['Arrival'] <= time)]
  rr_list_df = new_process_list.copy()
  while i < len(process_df):
    if rr_list_df.empty == False:
      for process in rr_list_df['Process_ID']:
        quantum = 0
        execution = rr_list_df.Execution_Time_Remaining[rr_list_df['Process_ID'] == process].item()
        if rr_list_df[rr_list_df['Process_ID'] == process]['Start'].item() == -1:
          rr_list_df.Start[rr_list_df['Process_ID'] == process] = time
          rr_list_df.Wait_Time[rr_list_df['Process_ID'] == process] = (time - rr_list_df.Arrival[rr_list_df['Process_ID'] == process].item())
        while execution > 0:
          execution -= 1
          time += 1
          quantum += 1
          if (time % 1 == 0): # In this case, if the polling time is up, we only concat new processes, we do not sort and break
            new_process_list = process_df.loc[(process_df['Arrival'] > (time-1)) & (process_df['Arrival'] <= time)]
            rr_list_df = pd.concat([rr_list_df,new_process_list])
          if (quantum % poll == 0): # However, if the process has been executing for poll seconds, then we:
            rr_list_df.Execution_Time_Remaining[rr_list_df['Process_ID'] == process] = execution # Save the process execution time remaining
            idx = rr_list_df.index.tolist() # Get the index of the current process list
            target = idx[0] # Save the top index to a temp variable
            idx.pop(0) # Remove the top index from the process list index
            rr_list_df = rr_list_df.reindex(idx+[target]) # "Move" the currently running row to the bottom of the dataframe by reindexing it to the very end
            break # And now we break to move back to the top of the outer while loop, so that the new process at the top of the dataframe can execute
        else:
          rr_list_df.Execution_Time_Remaining[rr_list_df['Process_ID'] == process] = process_df.Execution_Time_Remaining[process_df['Process_ID'] == process].item()
          rr_list_df.Finish[rr_list_df['Process_ID'] == process] = time
          rr_list_df.Turnaround_Time[rr_list_df['Process_ID'] == process] = (time - rr_list_df.Arrival[rr_list_df['Process_ID'] == process].item())
          if time <= rr_list_df.Deadline[rr_list_df['Process_ID'] == process].item():
              rr_list_df.Success[rr_list_df['Process_ID'] == process] = 1
          results_df = pd.concat([results_df,rr_list_df.loc[rr_list_df.Process_ID.eq(process)]])
          rr_list_df.drop(index=rr_list_df[rr_list_df['Process_ID'] == process].index, inplace = True)
          i += 1
        break
    else:
      time += 1
      if (time % poll == 0):
        new_process_list = process_df.loc[(process_df['Arrival'] > (time-poll)) & (process_df['Arrival'] <= time)]
        rr_list_df = new_process_list.copy()
  return results_df

In [ ]:
rr_results = rr(adhoc=transient_df)
rr_results

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
18,19,9,22,22,9,15,37,50,6,28,1
22,23,5,8,8,76,105,113,117,29,37,1
57,58,0,3,3,93,128,131,131,35,38,1
76,A,0,5,5,100,146,151,110,46,51,0
14,15,9,15,15,79,90,151,130,11,72,0
...,...,...,...,...,...,...,...,...,...,...,...
5,6,5,23,23,1905,1905,1950,1952,0,45,1
30,31,4,20,20,1909,1920,1955,1951,11,46,0
70,71,3,11,11,1965,1965,1976,1998,0,11,1
9,10,8,8,8,1983,1995,2003,2023,12,20,1


In [ ]:
print(list(rr_results['Process_ID']))
print(f'Wait times: {list(rr_results["Wait_Time"])}')
print(f'Average Wait times: {np.mean(list(rr_results["Wait_Time"]))}')
print(f'Turnaround times: {list(rr_results["Turnaround_Time"])}')
print(f'Average Turnaround times: {np.mean(list(rr_results["Turnaround_Time"]))}')
print(f'Successes: {list(rr_results["Success"])}')
print(f'Average Successes: {np.mean(list(rr_results["Success"]))}')

[19, 23, 58, 'A', 15, 76, 27, 61, 49, 'B', 7, 20, 39, 29, 'C', 'D', 50, 26, 38, 2, 8, 53, 5, 1, 37, 69, 44, 'E', 'F', 16, 67, 51, 22, 12, 59, 72, 'G', 57, 64, 55, 11, 11, 'H', 21, 34, 4, 56, 'I', 54, 3, 75, 'J', 46, 33, 65, 62, 60, 17, 28, 73, 45, 14, 41, 42, 52, 9, 32, 40, 68, 25, 13, 18, 70, 36, 74, 43, 24, 30, 66, 48, 35, 6, 31, 71, 10, 47]
Wait times: [6, 29, 35, 46, 11, 39, 28, 38, 5, 55, 5, 29, 31, 36, 0, 15, 13, 16, 25, 21, 8, 0, 0, 15, 2, 36, 8, 3, 18, 12, 34, 13, 24, 22, 17, 7, 2, 16, 19, 4, 0, 7, 10, 8, 12, 4, 4, 15, 1, 8, 20, 5, 4, 10, 18, 0, 15, 4, 14, 4, 8, 14, 8, 9, 10, 8, 6, 0, 13, 17, 4, 11, 14, 21, 11, 6, 3, 6, 28, 7, 20, 0, 11, 0, 12, 21]
Average Wait times: 13.651162790697674
Turnaround times: [28, 37, 38, 51, 72, 48, 85, 86, 8, 65, 65, 87, 85, 37, 50, 50, 24, 27, 70, 27, 13, 21, 19, 23, 54, 42, 61, 43, 48, 63, 36, 60, 71, 55, 53, 29, 52, 19, 28, 45, 6, 13, 50, 16, 24, 9, 45, 86, 28, 44, 21, 77, 5, 46, 42, 7, 28, 61, 70, 10, 18, 15, 18, 10, 46, 31, 30, 22, 15, 31, 45

In [ ]:
rr_results.loc[(rr_results['Arrival'] < 150) |
                  ((rr_results['Arrival'] >= 200) & (rr_results['Arrival'] < 250)) |
                  ((rr_results['Arrival'] >= 300) & (rr_results['Arrival'] < 350)) |
                  ((rr_results['Arrival'] >= 500) & (rr_results['Arrival'] < 550)) |
                  ((rr_results['Arrival'] >= 700) & (rr_results['Arrival'] < 750)) |
                  ((rr_results['Arrival'] >= 800) & (rr_results['Arrival'] < 850)) |
                  ((rr_results['Arrival'] >= 900) & (rr_results['Arrival'] < 950)) |
                  ((rr_results['Arrival'] >= 1000) & (rr_results['Arrival'] < 1050))]

,Process_ID,Priority,Duration,Execution_Time_Remaining,Arrival,Start,Finish,Deadline,Wait_Time,Turnaround_Time,Success
18,19,9,22,22,9,15,37,50,6,28,1
22,23,5,8,8,76,105,113,117,29,37,1
57,58,0,3,3,93,128,131,131,35,38,1
76,A,0,5,5,100,146,151,110,46,51,0
14,15,9,15,15,79,90,151,130,11,72,0
75,76,4,9,9,112,151,160,135,39,48,0
26,27,6,25,25,85,113,170,123,28,85,0
60,61,2,24,24,93,131,179,147,38,86,0
77,B,1,10,10,200,255,265,220,55,65,0
6,7,0,20,20,205,210,270,257,5,65,0


# Generate and Display Results

In [ ]:
def generate_results(iterations):
  algorithms = {'fcfs':fcfs,'priority':priority,'sjf':sjf,'least':least,'rr':rr} # Define a list of algorithms to test
  combined_results = pd.DataFrame(columns=['Min Wait Time', 'Mean Wait Time', 'Max Wait Time', 'Min Turnaround Time', 'Mean Turnaround Time', 'Max Turnaround Time', 'Min Successes', 'Mean Successes', 'Max Successes']) # Define a dataframe to store aggregated results
  for algorithm in algorithms: # For each algorithm:
    data = {'iteration':[],'mean_wait_time':[],'mean_turnaround_time':[],'mean_success':[]} # Define a temporary dictionary to store results
    for i in range(1,iterations+1): # For each iteration:
      results = algorithms[algorithm]() # Call the appropriate scheduling algorithm to get results
      data['iteration'].append(i) # Append the iteration count to the data dictionary
      data['mean_wait_time'].append(np.mean(results['Wait_Time'])) # Append the mean wait time to the data dictionary
      data['mean_turnaround_time'].append(np.mean(results['Turnaround_Time'])) # Append the mean turnaround time to the data dictionary
      data['mean_success'].append(np.mean(results["Success"])) # Append the mean results to the data dictionary
    line_fig = make_subplots(specs=[[{"secondary_y": True}]]) # Define a plotly subplots figure, WITH a secondary y axis, to be used for displaying both average times and successes
    line_fig.update_layout(yaxis1 = dict(range=[0, 100])) # Define the primary y axis
    line_fig.update_layout(width=600) # Set the width to something that can be reasonably fit in a document column
    line_fig.update_layout(yaxis2 = dict(range=[0, 1])) # Define the secondary y axis
    line_fig.add_trace(go.Scatter(x=data['iteration'], y=data['mean_wait_time'], name="Mean Wait Time"),secondary_y=False) # Add a scatter plot trace for the mean wait times
    line_fig.add_trace(go.Scatter(x=data['iteration'], y=data['mean_turnaround_time'], name="Mean Turnaround Time"),secondary_y=False) # Add a scatter plot trace for the mean turnaround times
    line_fig.add_trace(go.Scatter(x=data['iteration'], y=data['mean_success'], name="Mean Success"),secondary_y=True) # Add a scatter plot trace for the mean successes, on the secondary axis
    line_fig.update_layout(title_text=f"{algorithm} Mean Performance Results") # Add a title to the figure
    line_fig.update_xaxes(title_text="Iteration", dtick=1) # Add a title to the x axis
    line_fig.update_yaxes(title_text="<b>Logical Time</b>", secondary_y=False) # Add a title to the primary y axis
    line_fig.update_yaxes(title_text="<b>Percentage</b>", secondary_y=True) # Add a title to the secondary y axis
    line_fig.show() # Plot and show the line figure (the overarching subplot figure will add the lines to all the scatter plots)
    combined_results.loc[algorithm] = [np.min(data['mean_wait_time']),
                              np.mean(data['mean_wait_time']),
                              np.max(data['mean_wait_time']),
                              np.min(data['mean_turnaround_time']),
                              np.mean(data['mean_turnaround_time']),
                              np.max(data['mean_turnaround_time']),
                              np.min(data['mean_success']),
                              np.mean(data['mean_success']),
                              np.max(data['mean_success'])] # Add all the aggregated results to the combined dataframe
  return combined_results # Return the combined results dataframe



In [ ]:
overall_results = generate_results(10)

In [ ]:
overall_results

,Min Wait Time,Mean Wait Time,Max Wait Time,Min Turnaround Time,Mean Turnaround Time,Max Turnaround Time,Min Successes,Mean Successes,Max Successes
fcfs,17.144737,19.438158,22.947368,31.407895,34.676316,39.934211,0.486842,0.667105,0.763158
priority,12.750000,19.053947,35.842105,27.776316,37.965789,55.026316,0.657895,0.714474,0.828947
sjf,8.565789,11.907895,17.289474,27.210526,30.984211,40.697368,0.723684,0.822368,0.894737
least,11.750000,14.323684,17.447368,25.486842,30.034211,34.394737,0.710526,0.823684,0.921053
rr,10.736842,12.746053,16.578947,31.276316,35.073684,41.921053,0.500000,0.684211,0.789474


In [ ]:
results_fig = px.bar(overall_results, x=overall_results.index, y=['Min Wait Time', 'Mean Wait Time', 'Max Wait Time', 'Min Turnaround Time', 'Mean Turnaround Time', 'Max Turnaround Time'], barmode='group') # Define a bar chart, using the combined overal results above except Successes
results_fig.update_layout(title_text=f"<b>Mean Performance Results</b>") # Add a title to the figure
results_fig.update_xaxes(title_text="<b>Algorithm</b>") # Add a title to the x axis
results_fig.update_yaxes(title_text="<b>Mean Logical Time</b>") # Add a title to the primary y axis
results_fig.update_legends(title_text="Metric") # Add a title to the legend
results_fig.show()

In [ ]:
success_fig = px.bar(overall_results, x=overall_results.index, y=['Min Successes', 'Mean Successes', 'Max Successes'], barmode='group') # Define a bar chart, using the combined overall results above, but only Successes
success_fig.update_layout(title_text=f"<b>Deadline Success Percentages</b>") # Add a title to the figure
success_fig.update_xaxes(title_text="<b>Algorithm</b>") # Add a title to the x axis
success_fig.update_yaxes(title_text="<b>Mean Percentage</b>") # Add a title to the primary y axis
success_fig.update_legends(title_text="Metric") # Add a title to the legend
success_fig.show()

# Test Different Scheduling Rates

In [ ]:
def generate_polling_results(iterations):
  algorithms = {'fcfs':fcfs,'priority':priority,'sjf':sjf,'least':least,'rr':rr} # Define a set of algorithms to test
  polls = [5,10,15,20,25,30,60,90] # Define a set of logical polling/scheduling rates to test
  combined_results = {} # Create a combined results dictionary to store results dataframes
  for algorithm in algorithms: # For each algorithm:
    data = {'poll':[],'mean_wait_time':[],'mean_turnaround_time':[],'mean_success':[]} # Create a temporary data dictionary to store results
    for poll_rate in polls: # For each polling rate:
      data['poll'].append(poll_rate) # Append the polling rate
      mean_wait_time = [] # Define a temporary array to store mean wait times
      mean_turnaround_time = [] # Define a temporary array to store mean turnaround times
      mean_success = [] # Define a temporary array to store mean successes
      for i in range(0,iterations): # For each iteration:
        results = algorithms[algorithm](poll=poll_rate) # Generate results, using the current polling rate
        mean_wait_time.append(np.mean(results['Wait_Time'])) # Append the mean wait times to the appropriate array
        mean_turnaround_time.append(np.mean(results['Turnaround_Time'])) # Append the mean turnaround times to the appropriate array
        mean_success.append(np.mean(results["Success"])) # Append the mean successes to the appropriate array
      data['mean_wait_time'].append((np.mean(mean_wait_time))) # Once all the iterations have ran, average the average wait times across all iterations and append them to the data dictionary
      data['mean_turnaround_time'].append(np.mean(mean_turnaround_time)) # Average the average turnaround times and append them to the data dictionary
      data['mean_success'].append(np.mean(mean_success)) # Average the average successes and append them to the data dictionary
    line_fig = make_subplots(specs=[[{"secondary_y": True}]]) # Define a plotly subplots figure, WITH a secondary y axis, to be used for displaying both average times and successes
    line_fig.update_layout(yaxis1 = dict(range=[0, 100])) # Define the primary axis range
    line_fig.update_layout(yaxis2 = dict(range=[0, 1])) # Define the secondary axis range
    line_fig.update_layout(width=600)
    line_fig.add_trace(go.Scatter(x=data['poll'], y=data['mean_wait_time'], name="Mean Wait Time"),secondary_y=False) # Add a scatter plot trace for the mean wait times
    line_fig.add_trace(go.Scatter(x=data['poll'], y=data['mean_turnaround_time'], name="Mean Turnaround Time"),secondary_y=False) # Add a scatter plot trace for the mean turnaround times
    line_fig.add_trace(go.Scatter(x=data['poll'], y=data['mean_success'], name="Mean Success"),secondary_y=True) # Add a scatter plot trace for the mean successes
    line_fig.update_layout(title_text=f"{algorithm} Scheduling Rate Results") # Add a title to the figure
    line_fig.update_xaxes(title_text="Scheduling/Polling Window") # Add a title to the x axis
    line_fig.update_yaxes(title_text="<b>Logical Time</b>", secondary_y=False) # Add a title to the primary y axis
    line_fig.update_yaxes(title_text="<b>Percentage</b>", secondary_y=True) # Add a title to the secondary y axis
    line_fig.show() # Plot and show the line figure (the overarching subplot figure will add the lines to all the scatter plots)
    combined_results[algorithm] = pd.DataFrame.from_dict(data) # Convert the temporary data dictionary to a dataframe and add it to the combined results dictionary
  return combined_results # Return the combined results dictionary

In [ ]:
scheduling_results = generate_polling_results(10)

In [ ]:
scheduling_results['fcfs']

,poll,mean_wait_time,mean_turnaround_time,mean_success
0,5,12.444737,27.178947,0.796053
1,10,15.367105,30.260526,0.769737
2,15,21.096053,36.809211,0.675000
3,20,21.267105,36.272368,0.646053
4,25,21.688158,36.313158,0.594737
5,30,30.273684,45.752632,0.451316
6,60,49.123684,63.789474,0.134211
7,90,73.673684,88.822368,0.050000


In [ ]:
scheduling_results['priority']

,poll,mean_wait_time,mean_turnaround_time,mean_success
0,5,10.930263,29.459211,0.806579
1,10,16.031579,36.907895,0.747368
2,15,15.506579,34.711842,0.755263
3,20,21.186842,41.346053,0.659211
4,25,24.472368,42.603947,0.589474
5,30,27.901316,46.509211,0.534211
6,60,48.960526,65.190789,0.234211
7,90,71.530263,87.767105,0.097368


In [ ]:
scheduling_results['sjf']

,poll,mean_wait_time,mean_turnaround_time,mean_success
0,5,6.672368,28.481579,0.832895
1,10,9.938158,30.277632,0.826316
2,15,12.425000,31.413158,0.803947
3,20,16.784211,38.011842,0.709211
4,25,18.635526,38.147368,0.664474
5,30,21.339474,41.006579,0.619737
6,60,41.972368,59.618421,0.277632
7,90,61.850000,78.159211,0.169737


In [ ]:
scheduling_results['least']

,poll,mean_wait_time,mean_turnaround_time,mean_success
0,5,6.952632,23.397368,0.900000
1,10,11.375000,27.476316,0.865789
2,15,14.242105,30.755263,0.815789
3,20,16.965789,33.381579,0.753947
4,25,20.960526,36.709211,0.661842
5,30,25.122368,41.846053,0.582895
6,60,42.082895,57.767105,0.273684
7,90,62.822368,78.360526,0.157895


In [ ]:
scheduling_results['rr']

,poll,mean_wait_time,mean_turnaround_time,mean_success
0,5,7.115789,41.278947,0.638158
1,10,11.097368,38.534211,0.627632
2,15,16.221053,40.113158,0.560526
3,20,20.943421,42.502632,0.523684
4,25,26.182895,46.765789,0.471053
5,30,28.384211,43.578947,0.497368
6,60,51.601316,66.905263,0.192105
7,90,71.377632,86.748684,0.109211


# Deprecated Code

In [ ]:
def priority():
  wait_times = []
  turnaround_times = []
  successes = []
  i = 0
  arrival = 0
  time = 0
  process_df = generate_processes()
  new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
  priority_list_df = new_process_list.sort_values(by=['Priority','Arrival'], ascending=[True,True])
  #print(f'Retrieved {new_process_list.shape[0]} processes at arrival time {arrival}, there are {priority_list_df.shape[0]} total in the process list')
  while i < len(process_df):
    if priority_list_df.empty == False:
      for process in priority_list_df['Process_ID']:
        execution = priority_list_df[priority_list_df['Process_ID'] == process]['Execution_Time_Remaining'].item()
        #process_details = process_list[process_list['Process_ID'] == process]
        #print(process_details['Duration'].item)
        while execution > 0:
          execution -= 1
          time += 1
          if (time % 90 == 0):
            #print(f'Process {process} interrupted to check for new processes with {execution} cycles remaining')
            priority_list_df.Execution_Time_Remaining[priority_list_df['Process_ID'] == process] = execution
            arrival += 1
            new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
            for process in new_process_list['Process_ID']:
              new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
              #print(new_process_list.Arrival[new_process_list['Process_ID'] == process].item())
            priority_list_df = pd.concat([priority_list_df,new_process_list])
            priority_list_df = priority_list_df.sort_values(by=['Priority','Duration'], ascending=[False,True])
            #print(f'Retrieved {new_process_list.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {priority_list_df.shape[0]} total in the process list')
            #print(priority_list_df)
            break
        else:
          #print(f"Process {process} arrived at logical time {arrival_time} with a deadline of {(arrival_time + priority_list_df.Deadline[priority_list_df['Process_ID'] == process].item())} and finished processing at logical time {time}")
          if (time - priority_list_df.Arrival[priority_list_df['Process_ID'] == process].item()) <= priority_list_df.Deadline[priority_list_df['Process_ID'] == process].item():
              successes.append(1)
          else:
              successes.append(0)
          i += 1
          wait_times.append((time - priority_list_df.Duration[priority_list_df['Process_ID'] == process].item()))
          turnaround_times.append(time)
          priority_list_df.drop(index=priority_list_df[priority_list_df['Process_ID'] == process].index, inplace = True)
        break
    else:
      arrival += 1
      new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
      for process in new_process_list:
          new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
      priority_list_df = new_process_list.sort_values(by=['Priority','Duration'], ascending=[False,True])
      #print(f'Retrieved {priority_list_df.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {priority_list_df.shape[0]} total in the process list')
      #time += 1
  return wait_times, turnaround_times, successes

In [ ]:
wait_times = []
turnaround_times = []
total_wait_time = 0
for i in range(0,np.max(arrival_time)+1):
  process_list = retrieve_processes(i)
  priority_list_df = process_list.sort_values(by=['Priority','Duration'], ascending=[False,True])
  if priority_list_df.empty == False:
    print(f'Retrieved {priority_list_df.shape[0]} processes at arrival time {i}')
    print(priority_list_df.head(10))
    for process in priority_list_df['Process_ID']:
      process_details = priority_list_df[priority_list_df['Process_ID'] == process]
      wait_time = total_wait_time
    #  print(process_details['Duration'].item)
      for i in range(0,process_details['Duration'].item()):
        total_wait_time += 1
      wait_times.append(wait_time)
      turnaround_times.append(total_wait_time)
  i+= 1
print(wait_times)

In [ ]:
def sjf():
  wait_times = []
  turnaround_times = []
  successes = []
  i = 0
  arrival = 0
  time = 0
  process_df = generate_processes()
  new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
  shortest_list_df =  new_process_list.sort_values(by=['Duration','Priority'], ascending=[True,False])
  #print(f'Retrieved {new_process_list.shape[0]} processes at arrival time {arrival}, there are {shortest_list_df.shape[0]} total in the process list')
  while i < len(process_df):
    if shortest_list_df.empty == False:
      for process in shortest_list_df['Process_ID']:
        execution = shortest_list_df[shortest_list_df['Process_ID'] == process]['Execution_Time_Remaining'].item()
        #process_details = process_list[process_list['Process_ID'] == process]
        #print(process_details['Duration'].item)
        while execution > 0:
          execution -= 1
          time += 1
          if (time % 90 == 0):
            #print(f'Process {process} interrupted to check for new processes with {execution} cycles remaining')
            shortest_list_df.Execution_Time_Remaining[shortest_list_df['Process_ID'] == process] = execution
            arrival += 1
            new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
            for process in new_process_list['Process_ID']:
              new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
              #print(new_process_list.Arrival[new_process_list['Process_ID'] == process].item())
            shortest_list_df = pd.concat([shortest_list_df,new_process_list])
            shortest_list_df = shortest_list_df.sort_values(by=['Duration','Priority'], ascending=[True,False])
            #print(f'Retrieved {new_process_list.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {shortest_list_df.shape[0]} total in the process list')
            #print(shortest_list_df)
            break
        else:
          #print(f"Process {process} arrived at logical time {arrival_time} with a deadline of {(arrival_time + shortest_list_df.Deadline[shortest_list_df['Process_ID'] == process].item())} and finished processing at logical time {time}")
          if (time - shortest_list_df.Arrival[shortest_list_df['Process_ID'] == process].item()) <= shortest_list_df.Deadline[shortest_list_df['Process_ID'] == process].item():
              successes.append(1)
          else:
              successes.append(0)
          i += 1
          wait_times.append((time - shortest_list_df.Duration[shortest_list_df['Process_ID'] == process].item()))
          turnaround_times.append(time)
          shortest_list_df.drop(index=shortest_list_df[shortest_list_df['Process_ID'] == process].index, inplace = True)
        break
    else:
      arrival += 1
      new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
      for process in new_process_list:
          new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
      shortest_list_df = new_process_list.sort_values(by=['Duration','Priority'], ascending=[True,False])
      #print(f'Retrieved {new_process_list.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {shortest_list_df.shape[0]} total in the process list')
      #time += 1
  return wait_times, turnaround_times, successes

In [ ]:
sjf_wait_times, sjf_turnaround_times, sjf_success = sjf()
print(f'Wait times: {sjf_wait_times}')
print(f'Average Wait times: {np.mean(sjf_wait_times)}')
print(f'Turnaround times: {sjf_turnaround_times}')
print(f'Average Turnaround times: {np.mean(sjf_turnaround_times)}')
print(f'Successes: {sjf_success}')
print(f'Average Successes: {np.mean(sjf_success)}')

Wait times: [0, 5, 27, 33, 59, 87, 91, 96, 104, 112, 121, 130, 141, 154, 180, 188, 215, 244, 246, 270, 273, 278, 278, 296, 323, 325, 334, 360, 370, 382, 394, 397, 413, 429, 450, 456, 471, 492, 513, 540, 550, 571, 598, 630, 630, 639, 648, 660, 676, 693, 720, 739, 753, 779, 810, 824, 840, 858, 900, 911, 922, 935, 951, 990, 978, 1001, 1013, 1041, 1069, 1074, 1075, 1080, 1082, 1090, 1103, 1127]
Average Wait times: 542.5921052631579
Turnaround times: [5, 27, 33, 59, 87, 91, 96, 104, 112, 121, 130, 141, 154, 178, 188, 217, 244, 246, 260, 273, 278, 288, 296, 323, 325, 334, 350, 370, 382, 394, 407, 413, 429, 445, 456, 476, 492, 513, 539, 550, 572, 598, 626, 634, 639, 648, 660, 676, 693, 711, 739, 762, 779, 805, 824, 840, 863, 884, 911, 922, 935, 951, 969, 999, 1001, 1029, 1041, 1069, 1074, 1075, 1079, 1082, 1091, 1103, 1127, 1155]
Average Turnaround times: 557.7894736842105
Successes: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [ ]:
def least():
  wait_times = []
  turnaround_times = []
  successes = []
  i = 0
  arrival = 0
  time = 0
  process_df = generate_processes()
  new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
  least_list_df =  new_process_list.sort_values(by=['Execution_Time_Remaining','Priority'], ascending=[True,False])
  #print(f'Retrieved {new_process_list.shape[0]} processes at arrival time {arrival}, there are {least_list_df.shape[0]} total in the process list')
  while i < len(process_df):
    if least_list_df.empty == False:
      for process in least_list_df['Process_ID']:
        execution = least_list_df[least_list_df['Process_ID'] == process]['Execution_Time_Remaining'].item()
        #process_details = process_list[process_list['Process_ID'] == process]
        #print(process_details['Duration'].item)
        while execution > 0:
          execution -= 1
          time += 1
          if (time % 90 == 0):
            #print(f'Process {process} interrupted to check for new processes with {execution} cycles remaining')
            least_list_df.Execution_Time_Remaining[least_list_df['Process_ID'] == process] = execution
            arrival += 1
            new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
            for process in new_process_list['Process_ID']:
              new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
              #print(new_process_list.Arrival[new_process_list['Process_ID'] == process].item())
            least_list_df = pd.concat([new_process_list,least_list_df])
            least_list_df = least_list_df.sort_values(by=['Execution_Time_Remaining','Priority'], ascending=[True,False])
            #print(f'Retrieved {new_process_list.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {least_list_df.shape[0]} total in the process list')
            #print(least_list_df)
            break
        else:
          #print(f"Process {process} arrived at logical time {arrival_time} with a deadline of {(arrival_time + least_list_df.Deadline[least_list_df['Process_ID'] == process].item())} and finished processing at logical time {time}")
          if (time - least_list_df.Arrival[least_list_df['Process_ID'] == process].item()) <= least_list_df.Deadline[least_list_df['Process_ID'] == process].item():
              successes.append(1)
          else:
              successes.append(0)
          i += 1
          wait_times.append((time - least_list_df.Duration[least_list_df['Process_ID'] == process].item()))
          turnaround_times.append(time)
          least_list_df.drop(index=least_list_df[least_list_df['Process_ID'] == process].index, inplace = True)
        break
    else:
      arrival += 1
      new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
      for process in new_process_list:
          new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
      least_list_df = new_process_list.sort_values(by=['Execution_Time_Remaining','Priority'], ascending=[True,False])
      #print(f'Retrieved {new_process_list.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {least_list_df.shape[0]} total in the process list')
      #time += 1
  return wait_times, turnaround_times, successes

In [ ]:
def edf():
  wait_times = []
  turnaround_times = []
  successes = []
  i = 0
  arrival = 0
  time = 0
  process_df = generate_processes()
  new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
  edf_list_df =  new_process_list.sort_values(by=['Deadline','Priority'], ascending=[True,False])
  #print(f'Retrieved {new_process_list.shape[0]} processes at arrival time {arrival}, there are {edf_list_df.shape[0]} total in the process list')
  while i < len(process_df):
    if edf_list_df.empty == False:
      for process in edf_list_df['Process_ID']:
        execution = edf_list_df[edf_list_df['Process_ID'] == process]['Execution_Time_Remaining'].item()
        #process_details = process_list[process_list['Process_ID'] == process]
        #print(process_details['Duration'].item)
        while execution > 0:
          execution -= 1
          time += 1
          if (time % 90 == 0):
            #print(f'Process {process} interrupted to check for new processes with {execution} cycles remaining')
            edf_list_df.Execution_Time_Remaining[edf_list_df['Process_ID'] == process] = execution
            arrival += 1
            new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
            #print(f'New processes: {new_process_list}')
            for process in new_process_list['Process_ID']:
              new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
              deadline = new_process_list.Deadline[new_process_list['Process_ID'] == process].item() + time
              new_process_list.Deadline[new_process_list['Process_ID'] == process] = deadline
              #print(new_process_list.Arrival[new_process_list['Process_ID'] == process].item())
            edf_list_df = pd.concat([new_process_list,edf_list_df])
            edf_list_df = edf_list_df.sort_values(by=['Deadline','Priority'], ascending=[True,False])
            #print(f'Retrieved {new_process_list.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {edf_list_df.shape[0]} total in the process list')
            #print(f'Sorted processes: {edf_list_df}')
            break
        else:
          #print(f"Process {process} arrived at logical time {edf_list_df.Arrival[edf_list_df['Process_ID'] == process].item()} with a deadline of {edf_list_df.Deadline[edf_list_df['Process_ID'] == process].item()} and finished processing at logical time {time}")
          if time <= edf_list_df.Deadline[edf_list_df['Process_ID'] == process].item():
              successes.append(1)
          else:
              successes.append(0)
          i += 1
          wait_times.append((time - edf_list_df.Duration[edf_list_df['Process_ID'] == process].item()))
          turnaround_times.append(time)
          edf_list_df.drop(index=edf_list_df[edf_list_df['Process_ID'] == process].index, inplace = True)
        break
    else:
      arrival += 1
      new_process_list = process_df.loc[process_df['Arrival_Group'] == arrival]
      for process in new_process_list['Process_ID']:
        new_process_list.Arrival[new_process_list['Process_ID'] == process] = time
        deadline = new_process_list.Deadline[new_process_list['Process_ID'] == process].item() + time
        new_process_list.Deadline[new_process_list['Process_ID'] == process] = deadline
      edf_list_df = new_process_list.sort_values(by=['Deadline','Priority'], ascending=[True,False])
      #print(f'Retrieved {new_process_list.shape[0]} processes at logical time {time} and arrival group {arrival}, there are {edf_list_df.shape[0]} total in the process list')
      #time += 1
  return wait_times, turnaround_times, successes